In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def tokenize(sentence: str):
    tokens = sentence.lower().split()
    tokens = [process_token(t) for t in tokens]
    return tokens

def process_token(token: str):
    if token.strip() == "/":
        return token
    return re.sub(r'[^\w\s]', '', token.strip())

def vectorize(tokens):
    vocab, index = {}, 1
    vocab['<pad>'] = 0
    for token in tokens:
        token = token.strip()
        if token not in vocab:
            vocab[token] = index
            index += 1
    return vocab

def find_max_length(vectorized_poems):
    max_length = 0
    for poem in vectorized_poems:
        max_length = max(max_length, len(poem))   
    return max_length

In [ ]:
all_text = " ".join(data["processed_title"].to_list())
tokens = tokenize(all_text)
vocab_map = vectorize(tokens)

In [73]:
x = haikus[:, :19-1]  # input consists of all words except the last
y = haikus[:, 1:]

In [74]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [75]:
# training_examples = []
# window_size = 4
# for haiku in haikus:
#     for i in range(len(haiku) - window_size + 1):
#         input_words = haiku[i:i+window_size-1]
#         output_word = haiku[i+window_size-1]
#         training_examples.append((input_words, output_word))
# training_examples[:20]

## Transformer

In [76]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Embedding, Dense, Dropout, LayerNormalization
from keras.layers import MultiHeadAttention, TimeDistributed, Add, Concatenate
from keras.models import Model

In [77]:
# Encoder
encoder_inputs = Input(shape=(16,), name='encoder_inputs')
embedding_dim = 32
hidden_dim = 256
x = Embedding(len(vocab_map)+1, embedding_dim)(encoder_inputs)
# Self-attention layer
attn_output = MultiHeadAttention(num_heads=4, key_dim=embedding_dim)(x, x)
x = Add()([x, attn_output])
x = LayerNormalization(epsilon=1e-6)(x)
# Feed-forward layer
ff_output = TimeDistributed(Dense(hidden_dim, activation='relu'))(x)
ff_output = TimeDistributed(Dense(embedding_dim))(ff_output)
x = Add()([x, ff_output])
encoder_outputs = LayerNormalization(epsilon=1e-6)(x)
# -----
# Decoder
decoder_inputs = Input(shape=(15,), name='decoder_inputs')
x = Embedding(len(vocab_map)+1, embedding_dim)(decoder_inputs)
# Self-attention layer
attn_output1 = MultiHeadAttention(num_heads=4, key_dim=embedding_dim)(x, x)
x = Add()([x, attn_output1])
x = LayerNormalization(epsilon=1e-6)(x)
# Encoder-decoder attention layer
attn_output2 = MultiHeadAttention(num_heads=4, key_dim=embedding_dim)(x, encoder_outputs)
x = Add()([x, attn_output2])
x = LayerNormalization(epsilon=1e-6)(x)
decoder_outputs = TimeDistributed(Dense(len(vocab_map)+1, activation='softmax'))(x)

In [78]:
# Define the model
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

In [79]:
x = haikus[:, :-3]
y = haikus[:, 3:]

In [80]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [81]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [107]:
model.fit(x=[x_train, y_train[:, :-1]], y=y_train[:, 1:], batch_size=256, epochs=5)

Epoch 1/5
48/48 [==============================] - 47s 974ms/step - loss: 6.7841
Epoch 2/5
48/48 [==============================] - 47s 986ms/step - loss: 5.5397
Epoch 3/5
48/48 [==============================] - 47s 983ms/step - loss: 5.0182
Epoch 4/5
48/48 [==============================] - 47s 984ms/step - loss: 4.7290
Epoch 5/5
48/48 [==============================] - 47s 989ms/step - loss: 4.4760


In [84]:
vocab_map_inv = dict([(value, key) for key, value in vocab_map.items()])

In [105]:
def generate_poem(input_words: list):
    window_size = 3
    while len(input_words) < window_size - 1:
        input_words.insert(0, "<pad>")
    vectorized_input = [vocab_map[word] for word in input_words]
    print(f"User specified words {input_words} which were vectorized as {vectorized_input}")
    output_poem = input_words
    
    for i in range(13):
        print(i)
        input = np.array([np.pad(vectorized_input, (0, 16 - len(vectorized_input)), constant_values=len(vocab_map))])
        prediction = model.predict([input, np.zeros((1, 15))], verbose = 0)
        new_word_vector = np.argmax(prediction[0, i])
        vectorized_input.append(new_word_vector)
        new_word = vocab_map_inv[new_word_vector]
        output_poem.append(new_word)
    output = " ".join(output_poem)
    print(f"OUTPUT POEM: {output}")

In [106]:
generate_poem(["hello", "i", "am"])

User specified words ['hello', 'i', 'am'] which were vectorized as [220, 9, 153]
0
1
2
3
4
5
6
7
8
9
10
11
12
OUTPUT POEM: hello i am <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
